In [1]:
import pandas as pd
import altair as alt
# alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

# Evaluate

In [3]:
df = pd.read_csv('../javascript/JAN-10-2024/gov_aggregated_results_gov.csv', header=None, names=['page_id', 'issue_id', 'violations', 'passes'])

# df = df[df.issue_id == 'image-alt']

df.head(10)
# len(df.issue_id.unique().tolist())

,page_id,issue_id,violations,passes
0,0,meta-refresh,1,0
1,14,button-name,1,5
2,14,image-alt,3,28
3,13,image-alt,1,15
4,13,aria-valid-attr-value,0,26
5,3,image-alt,1,7
6,9,image-alt,2,22
7,9,label,1,12


$$
A3 = 1 - \Pi_b (1 - F_b)^{\frac{B_pb}{N_pb} + \frac{B_pb}{B_p}}
$$

> Equation 3 presents the formula for computing the A3 metric, where Bpb is the total of actual points of failure of a checkpoint b in page p, b is the barrier (checkpoint violation), Npb is the total of potential points of failure of a checkpoint b in page p, and Fb identifies the severity of a certain barrier b (this weight is calculated by simple heuristics, by combining the results of an automatic evaluation and manual testing or by disabled users feedback [221). The authors of this metric performed an experimental study to compare the results between A3 and UWEM and understand the differences between them. A checkpoint weight of 0.05 was used for all checkpoints, assuming that all of them would have the same importance. This experiment was conducted with a group of six disabled users that evaluated six web pages. After applying both metrics, the authors concluded that A3 outperformed UWEM in the experiment [11].

In [18]:
_df = df.merge(df.groupby(['page_id'])['violations'].sum().reset_index().rename(columns={'violations':'Bp'}), on='page_id', how='left')

_df['Npb'] = _df['violations'] + _df['passes']
_df['Bpb_over_Npb'] = _df['violations'] / _df['Npb']
_df['Bpb_over_Bb'] = _df['violations'] / _df['Bp']
_df['Fb'] = 0.1

_df = _df.merge(_df.groupby(['page_id'])['Npb'].sum().reset_index().rename(columns={'Npb':'Np'}), on='page_id', how='left')

_df['A3'] = 1 - _df['Fb']
_df['A3'] = _df['A3'] ** (_df['Bpb_over_Npb'] + _df['Bpb_over_Bb'])
_df.drop(['issue_id'], inplace=True, axis=1)
_df = _df[['page_id', 'A3']].groupby(['page_id']).prod().reset_index()
_df['A3'] = 1 - _df['A3']

_df

,page_id,A3
0,0,0.190000
1,3,0.111775
2,9,0.115069
3,13,0.105907
4,14,0.124637


In [21]:
# or just ratio
_df = df.copy()

_df.drop(['issue_id'], axis=1, inplace=True)

_df = _df.groupby(['page_id']).sum().reset_index()

_df['Np'] = _df['violations'] + _df['passes']
_df['violation_ratio'] = _df['violations'] / _df['Np']
# _df['Bpb_over_Npb'] = _df['violations'] / _df['Npb']
# _df['Bpb_over_Bb'] = _df['violations'] / _df['Bp']
# _df['Fb'] = 0.1

# _df = _df.merge(_df.groupby(['page_id'])['Npb'].sum().reset_index().rename(columns={'Npb':'Np'}), on='page_id', how='left')

# _df['A3'] = 1 - _df['Fb']
# _df['A3'] = _df['A3'] ** (_df['Bpb_over_Npb'] + _df['Bpb_over_Bb'])
# _df.drop(['issue_id'], inplace=True, axis=1)
# _df = _df[['page_id', 'A3']].groupby(['page_id']).prod().reset_index()
# _df['A3'] = 1 - _df['A3']

_df

,page_id,violations,passes,Np,violation_ratio
0,0,1,0,1,1.000000
1,3,1,7,8,0.125000
2,9,3,34,37,0.081081
3,13,1,41,42,0.023810
4,14,4,33,37,0.108108


In [ ]:
_df['id'] = _df['page_id'].apply(lambda x: x.split('_')[0])
df_meta = pd.read_csv(f'../output/Nov-21-2023/data-portal_metadata.csv')

_df['id'] = _df['id'].astype(str)
df_meta['id'] = df_meta['id'].astype(str)

_df = _df.merge(df_meta, left_on='id', right_on='id', how='left')
_df

In [6]:
_df.columns

Index(['page_id', 'violations', 'passes', 'Np', 'violation_ratio'], dtype='object')

In [11]:
alt.Chart(
    _df
).mark_tick(
    # opacity=0.01
).encode(
    alt.X(f'Np:Q'),
    # alt.Y('country:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)

In [13]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_tick(
    # opacity=0.01
).encode(
    alt.X(f'violations:Q'),
    # alt.Y('country:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)

In [25]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'violation_ratio:Q', title='The ratio of violations among all potential violations').axis(format='.1%').scale(alt.Scale(domain=(0, 1))),
    alt.Y('page_id:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)
# https://www.congress.gov/

alt.Chart(...)

In [19]:
alt.Chart(
    _df #[_df.country.isin(COUNTRIES)]
).mark_bar(
    # opacity=0.01
).encode(
    alt.X(f'A3:Q'),
    alt.Y('page_id:N', sort='-x'),
    # alt.Color('count()'),
    # alt.Tooltip(['count()'])
).properties(
    # title='Countries with more than 10 sites',
    width=500
)

alt.Chart(...)